# 수정 포인트
1. 데이터 증강 추가, 기존에는 정규화만 있었음
2. Early Stopping 추가

# Import

In [1]:
import os
import random

import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import log_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Hyperparameter Setting

In [2]:
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 64,
    'EPOCHS': 50,
    'LEARNING_RATE': 1e-4,
    'SEED' : 42
}

# Fixed RandomSeed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

# CustomDataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if is_test:
            # 테스트셋: 라벨 없이 이미지 경로만 저장
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith(('.jpg')):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            # 학습셋: 클래스별 폴더 구조에서 라벨 추출
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith(('.jpg')):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = np.array(Image.open(img_path).convert('RGB'))
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        else:
            img_path, label = self.samples[idx]
            image = np.array(Image.open(img_path).convert('RGB'))
            if self.transform:
                image = self.transform(image=image)['image']
            return image, label


# Data Load

In [5]:
train_root = './data/train'
test_root = './data/test'

In [6]:
# 학습 데이터에 적용할 데이터 증강 파이프라인
train_transform = A.Compose([
    # 이미지 크기를 모델 입력 크기에 맞게 고정
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),

    # 좌우 반전: 차량이 좌우 어느 방향을 향해 있어도 학습 가능하도록
    A.HorizontalFlip(p=0.5),

    # 밝기/대비 랜덤 조절: 야외 촬영 시 조명 차이를 반영
    A.RandomBrightnessContrast(p=0.2),

    # 색조(Hue), 채도(Saturation), 명도(Value) 변경: 색상에 덜 민감하게
    A.HueSaturationValue(p=0.2),

    # 이미지 이동, 확대/축소, 회전: 다양한 촬영 각도 및 위치 대응
    A.ShiftScaleRotate(shift_limit=0.05,   # 최대 ±5% 이동
                       scale_limit=0.05,   # 최대 ±5% 확대/축소
                       rotate_limit=15,    # 최대 ±15도 회전
                       p=0.5),

    # 그림자 효과 추가: 자연광 환경에서의 촬영 상황을 반영
    A.RandomShadow(p=0.2),

    # RGB 채널별 색상 이동: 다양한 카메라 환경, 화이트밸런스 차이 대응
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.2),

    # 이미지 일부분을 무작위로 지우기 (일부 가림 상황을 반영)
    A.CoarseDropout(max_holes=2,          # 최대 2개 영역 제거
                    max_height=16, 
                    max_width=16, 
                    p=0.3),

    # 픽셀값을 정규화 (ImageNet 사전학습 모델 기준 평균/표준편차)
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),

    # numpy 이미지 → PyTorch Tensor 변환
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\user\AppData\Local\Temp\ipykernel_10848\2295189183.py:28: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=2,          # 최대 2개 영역 제거


In [7]:
# 전체 데이터셋 로드
full_dataset = CustomImageDataset(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


# Model Define

In [8]:
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet18(pretrained=True)  # ResNet18 모델 불러오기
        self.feature_dim = self.backbone.fc.in_features 
        self.backbone.fc = nn.Identity()  # feature extractor로만 사용
        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기

    def forward(self, x):
        x = self.backbone(x)       
        x = self.head(x) 
        return x

# Train/ Validation

In [9]:
model = BaseModel(num_classes=len(class_names)).to(device)
best_logloss = float('inf')
# 추가: Early Stopping을 위한 변수
patience = 5  # 개선 없을 때 몇 epoch까지 기다릴지
counter = 0   # 현재까지 개선되지 않은 횟수

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])

# 학습 및 검증 루프
for epoch in range(CFG['EPOCHS']):
    # Train
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)  # logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))

    # 결과 출력
    print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.4f}%")

    # Best model 저장
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        torch.save(model.state_dict(), f'best_model.pth')
        print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.4f})")
        counter = 0  # 성능 개선되었으므로 초기화
    else:
        counter += 1
        print(f"⚠️ No improvement for {counter} epoch(s).")
        if counter >= patience:
            print(f"⏹ Early stopping triggered at epoch {epoch+1}")
            break  # 학습 중단

c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[Epoch 1/10] Validation: 100%|██████████| 104/104 [00:41<00:00,  2.53it/s]
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 4.6025 || Valid Loss : 2.7474 | Valid Accuracy : 58.3283%
📦 Best model saved at epoch 1 (logloss: 2.7463)


[Epoch 2/10] Validation: 100%|██████████| 104/104 [00:41<00:00,  2.52it/s]
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 1.9825 || Valid Loss : 1.0552 | Valid Accuracy : 81.6083%
📦 Best model saved at epoch 2 (logloss: 1.0550)


[Epoch 3/10] Validation: 100%|██████████| 104/104 [00:35<00:00,  2.93it/s]
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


...

[Epoch 9/10] Validation: 100%|██████████| 104/104 [00:36<00:00,  2.87it/s]
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Train Loss : 0.1219 || Valid Loss : 0.2125 | Valid Accuracy : 94.1008%
📦 Best model saved at epoch 9 (logloss: 0.2126)


[Epoch 10/10] Validation: 100%|██████████| 104/104 [00:35<00:00,  2.94it/s]

Train Loss : 0.1082 || Valid Loss : 0.1944 | Valid Accuracy : 94.1762%
📦 Best model saved at epoch 10 (logloss: 0.1948)



c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\sklearn\metrics\_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


# Inference

In [10]:
test_dataset = CustomImageDataset(test_root, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [11]:
# 저장된 모델 로드
model = BaseModel(num_classes=len(class_names))
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

# 추론
model.eval()
results = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        # 각 배치의 확률을 리스트로 변환
        for prob in probs.cpu():  # prob: (num_classes,)
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Submission

In [12]:
submission = pd.read_csv('./result/sample_submission.csv', encoding='utf-8-sig')

# 'ID' 컬럼을 제외한 클래스 컬럼 정렬
class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('./result/submission_02.csv', index=False, encoding='utf-8-sig')